In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split as tts
import lightgbm as lgb
import optuna

In [2]:
df_train = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv',low_memory=False,index_col='isic_id')
df_test = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv',low_memory=False,index_col='isic_id')

In [3]:
df_train = df_train.drop(['patient_id'],axis=1)
df_test = df_test.drop(['patient_id'],axis=1)

In [4]:
def reduce_memory_usage(df, verbose=True):
    start_memory = df.memory_usage().sum() / 1024**2
    print("Memory usage of df is {:.2f} MB".format(start_memory))
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:  # float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    
    end_memory = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_memory))
    print("Decreased memory percentage is: {:.2f}%".format(100 * (start_memory - end_memory) / start_memory))

    return df

reduce_memory_usage(df_train)
reduce_memory_usage(df_test)

Memory usage of df is 165.23 MB
Memory usage after optimization is: 79.94 MB
Decreased memory percentage is: 51.62%
Memory usage of df is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased memory percentage is: 59.59%


,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license
isic_id,,,,,,,,,,,,,,,,,,,,,
ISIC_0015657,45.0,male,posterior torso,2.699219,TBP tile: close-up,3D: XP,22.796875,20.00000,28.390625,27.046875,...,0.304932,1.281250,2.300781,0.479248,20,-155.1250,1511.0,114.000000,Memorial Sloan Kettering Cancer Center,CC-BY
ISIC_0015729,35.0,female,lower extremity,2.519531,TBP tile: close-up,3D: XP,16.656250,9.65625,31.312500,27.531250,...,0.000000,1.271484,2.011719,0.426270,25,-112.3750,629.5,-15.015625,"Frazer Institute, The University of Queensland...",CC-BY
ISIC_0015740,65.0,male,posterior torso,3.160156,TBP tile: close-up,3D: XP,24.250000,19.93750,30.468750,28.390625,...,0.230713,1.080078,2.705078,0.365967,110,-84.3125,1304.0,-28.578125,FNQH Cairns,CC-BY


In [5]:
from sklearn.metrics import roc_auc_score
def score(solution: np.ndarray, submission: np.ndarray, min_tpr: float=0.80) -> float:
    v_gt = abs(solution-1)
    v_pred = np.array([1.0 - x for x in submission])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)   
    return(partial_auc)
def pauc_80(true, pred):
    score_value = score(true, pred, min_tpr=0.8)
    return 'pauc_80', score_value, True

In [6]:
feature_col = list(set(df_train) & set(df_test))

In [7]:
categorical_col = [i for i in feature_col if df_train[i].dtype=='object']

In [8]:
X = df_train[feature_col]
y = df_train['target']

In [9]:
cat_idx = [X.columns.get_loc(i) for i in categorical_col]
for col in categorical_col:
    X[col] = X[col].astype('category')
    df_test[col] = df_test[col].astype('category')

In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401059 entries, ISIC_0015670 to ISIC_9999967
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   tbp_lv_B                     401059 non-null  float16 
 1   tbp_lv_A                     401059 non-null  float16 
 2   tbp_lv_C                     401059 non-null  float16 
 3   tbp_lv_x                     401059 non-null  float16 
 4   tbp_lv_deltaLBnorm           401059 non-null  float16 
 5   anatom_site_general          395303 non-null  category
 6   tbp_lv_H                     401059 non-null  float16 
 7   tbp_lv_deltaL                401059 non-null  float16 
 8   tbp_lv_L                     401059 non-null  float16 
 9   tbp_lv_deltaLB               401059 non-null  float16 
 10  tbp_lv_norm_color            401059 non-null  float16 
 11  tbp_lv_location_simple       401059 non-null  category
 12  age_approx                   398

In [11]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=7)

In [12]:
def objective(trial):
    param_grid = {
        "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }
    acc = np.zeros(5)
    model = lgb.LGBMClassifier(**param_grid,objective='binary',n_jobs=-1,random_state=42,verbosity=-1)
    for idx, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_t, X_v = X.iloc[train_idx], X.iloc[test_idx]
        y_t, y_v = y[train_idx], y[test_idx]
        eval_set = [(X_v,y_v)]
        model.fit(X_t,y_t,
        categorical_feature=cat_idx,  # Specify the categoricals
        eval_set=eval_set,
        callbacks=[lgb.early_stopping(200),lgb.log_evaluation(40)],
        eval_metric=[pauc_80],)
        y_pred = model.predict_proba(X_v)[:,1]
        acc[idx] = score(y_v,y_pred)
    return np.mean(acc)

In [13]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=100)

[I 2024-08-17 16:35:24,978] A new study created in memory with name: no-name-71264b5e-7ea8-495d-b4a2-e0d26efd3280
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 200 rounds
[40]	valid_0's binary_logloss: 0.00771645	valid_0's pauc_80: 0.02
[80]	valid_0's binary_logloss: 0.00771645	valid_0's pauc_80: 0.02
[120]	valid_0's binary_logloss: 0.00771645	valid_0's pauc_80: 0.02
[160]	valid_0's binary_logloss: 0.00771645	valid_0's pauc_80: 0.02
[200]	valid_0's binary_logloss: 0.00771645	valid_0's pauc_80: 0.02
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.00771645	valid_0's pauc_80: 0.02
Training until validation scores don't improve for 200 rounds
[40]	valid_0's binary_logloss: 0.0078028	valid_0's pauc_80: 0.02
[80]	valid_0's binary_logloss: 0.0078028	valid_0's pauc_80: 0.02
[120]	valid_0's binary_logloss: 0.0078028	valid_0's pauc_80: 0.02
[160]	valid_0's binary_logloss: 0.0078028	valid_0's pauc_80: 0.02
[200]	valid_0's binary_logloss: 0.0078028	valid_0's pauc_80: 0.02
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.0078028	valid_0's pauc_80: 0.02
Training until v

[I 2024-08-17 16:36:31,257] Trial 0 finished with value: 0.01999999999999999 and parameters: {'device_type': 'gpu', 'n_estimators': 10000, 'learning_rate': 0.20722789689940094, 'num_leaves': 2060, 'max_depth': 3, 'min_data_in_leaf': 3200, 'lambda_l1': 40, 'lambda_l2': 30, 'min_gain_to_split': 6.597700090544639, 'bagging_fraction': 0.2, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 0 with value: 0.01999999999999999.


[200]	valid_0's binary_logloss: 0.00771653	valid_0's pauc_80: 0.02
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.00771653	valid_0's pauc_80: 0.02
Training until validation scores don't improve for 200 rounds
[40]	valid_0's binary_logloss: 0.00771645	valid_0's pauc_80: 0.02
[80]	valid_0's binary_logloss: 0.00771645	valid_0's pauc_80: 0.02
[120]	valid_0's binary_logloss: 0.00771645	valid_0's pauc_80: 0.02
[160]	valid_0's binary_logloss: 0.00765208	valid_0's pauc_80: 0.0433156
[200]	valid_0's binary_logloss: 0.00765208	valid_0's pauc_80: 0.0433156
[240]	valid_0's binary_logloss: 0.00765208	valid_0's pauc_80: 0.0433156
[280]	valid_0's binary_logloss: 0.00765208	valid_0's pauc_80: 0.0433156
[320]	valid_0's binary_logloss: 0.00765208	valid_0's pauc_80: 0.0433156
Early stopping, best iteration is:
[134]	valid_0's binary_logloss: 0.00768381	valid_0's pauc_80: 0.0433156
Training until validation scores don't improve for 200 rounds
[40]	valid_0's binary_logloss: 0.0078028	va

[I 2024-08-17 16:38:17,973] Trial 1 finished with value: 0.053271295803490736 and parameters: {'device_type': 'gpu', 'n_estimators': 10000, 'learning_rate': 0.1773905482119077, 'num_leaves': 1620, 'max_depth': 12, 'min_data_in_leaf': 1800, 'lambda_l1': 50, 'lambda_l2': 45, 'min_gain_to_split': 10.168355395238331, 'bagging_fraction': 0.4, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 1 with value: 0.053271295803490736.


...

Training until validation scores don't improve for 200 rounds
[40]	valid_0's binary_logloss: 0.00572508	valid_0's pauc_80: 0.141963
[80]	valid_0's binary_logloss: 0.00559236	valid_0's pauc_80: 0.146752
[120]	valid_0's binary_logloss: 0.00555994	valid_0's pauc_80: 0.14856
[160]	valid_0's binary_logloss: 0.00554553	valid_0's pauc_80: 0.148882
[200]	valid_0's binary_logloss: 0.00552561	valid_0's pauc_80: 0.149653
[240]	valid_0's binary_logloss: 0.00552202	valid_0's pauc_80: 0.149925
[280]	valid_0's binary_logloss: 0.00551902	valid_0's pauc_80: 0.150082
[320]	valid_0's binary_logloss: 0.00551485	valid_0's pauc_80: 0.150038
[360]	valid_0's binary_logloss: 0.00551023	valid_0's pauc_80: 0.150369
[400]	valid_0's binary_logloss: 0.00550767	valid_0's pauc_80: 0.150576
[440]	valid_0's binary_logloss: 0.00550507	valid_0's pauc_80: 0.1506
[480]	valid_0's binary_logloss: 0.0055062	valid_0's pauc_80: 0.150687
[520]	valid_0's binary_logloss: 0.00550276	valid_0's pauc_80: 0.15097
[560]	valid_0's binary

[I 2024-08-17 21:00:37,940] Trial 98 finished with value: 0.15684067385210074 and parameters: {'device_type': 'gpu', 'n_estimators': 10000, 'learning_rate': 0.28913175227237686, 'num_leaves': 1800, 'max_depth': 10, 'min_data_in_leaf': 4200, 'lambda_l1': 15, 'lambda_l2': 85, 'min_gain_to_split': 0.32768955185895576, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.5}. Best is trial 66 with value: 0.16597482738778915.


Training until validation scores don't improve for 200 rounds
[40]	valid_0's binary_logloss: 0.00646967	valid_0's pauc_80: 0.12114
[80]	valid_0's binary_logloss: 0.00643066	valid_0's pauc_80: 0.121647
[120]	valid_0's binary_logloss: 0.00641262	valid_0's pauc_80: 0.121698
[160]	valid_0's binary_logloss: 0.00636732	valid_0's pauc_80: 0.123854
[200]	valid_0's binary_logloss: 0.00633758	valid_0's pauc_80: 0.124097
[240]	valid_0's binary_logloss: 0.00631931	valid_0's pauc_80: 0.124299
[280]	valid_0's binary_logloss: 0.00631931	valid_0's pauc_80: 0.124299
[320]	valid_0's binary_logloss: 0.00631931	valid_0's pauc_80: 0.124299
[360]	valid_0's binary_logloss: 0.00631931	valid_0's pauc_80: 0.124299
[400]	valid_0's binary_logloss: 0.00631931	valid_0's pauc_80: 0.124299
Early stopping, best iteration is:
[238]	valid_0's binary_logloss: 0.00631931	valid_0's pauc_80: 0.124299
Training until validation scores don't improve for 200 rounds
[40]	valid_0's binary_logloss: 0.00643427	valid_0's pauc_80: 0.

[I 2024-08-17 21:02:41,424] Trial 99 finished with value: 0.13816388586754433 and parameters: {'device_type': 'gpu', 'n_estimators': 10000, 'learning_rate': 0.2627504114047978, 'num_leaves': 1140, 'max_depth': 12, 'min_data_in_leaf': 1000, 'lambda_l1': 20, 'lambda_l2': 100, 'min_gain_to_split': 11.54894902900742, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.6000000000000001}. Best is trial 66 with value: 0.16597482738778915.


Early stopping, best iteration is:
[358]	valid_0's binary_logloss: 0.00628096	valid_0's pauc_80: 0.146707


In [14]:
final_model = lgb.LGBMClassifier(**study.best_params,objective='binary',n_jobs=-1,random_state=42,verbosity=-1)
final_model.fit(X,y,categorical_feature=cat_idx)

LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, device_type='gpu',
               feature_fraction=0.6000000000000001, lambda_l1=5, lambda_l2=95,
               learning_rate=0.27831684670109436, max_depth=11,
               min_data_in_leaf=2000, min_gain_to_split=0.06709112740017042,
               n_estimators=10000, n_jobs=-1, num_leaves=1200,
               objective='binary', random_state=42, verbosity=-1)

In [15]:
study.best_params

{'device_type': 'gpu',
 'n_estimators': 10000,
 'learning_rate': 0.27831684670109436,
 'num_leaves': 1200,
 'max_depth': 11,
 'min_data_in_leaf': 2000,
 'lambda_l1': 5,
 'lambda_l2': 95,
 'min_gain_to_split': 0.06709112740017042,
 'bagging_fraction': 0.8,
 'bagging_freq': 1,
 'feature_fraction': 0.6000000000000001}

In [16]:
X_test = df_test
sub = pd.DataFrame({'target':final_model.predict_proba(X_test)[:,1]},index=df_test.index)
sub.to_csv('submission.csv')
!head submission.csv

isic_id,target
ISIC_0015657,0.00839364948213531
ISIC_0015729,0.004681656894927234
ISIC_0015740,0.0035049111705586517
